In [ ]:
import torch
import torchvision
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets
from torch import optim
from tqdm import tqdm
from torchvision.transforms import ToTensor
from torchsummary import summary

In [ ]:
training=datasets.CIFAR10(root="data",train=True,download=True,transform=ToTensor())
testing=datasets.CIFAR10(root="data",train=False,download=True,transform=ToTensor())

In [ ]:
train_dataloader=DataLoader(training,batch_size=64)
test_dataloader=DataLoader(testing,batch_size=64)

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.NIN=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=5,padding=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=32,kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=3,kernel_size=1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=5,padding=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=32,kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=3,kernel_size=1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=5,padding=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=32,kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=3,kernel_size=1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool2d(kernel_size=2,stride=2)

            )
        self.flatten=nn.Flatten()
        self.l=nn.Linear(48,10)
    def forward(self,x):
        x=self.NIN(x)
        x=self.flatten(x)
        x=self.l(x)
        return x





In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=NN().to(device)

In [ ]:
summary(model,input_size=(3,32,32))

In [ ]:
learning_rate=0.001

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer=SummaryWriter('tensor/CIFAR10x')

In [ ]:
loss_fun=nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
def accuracy_check(loader,model):
    correct=0
    total=0
    model.eval()
    with torch.no_grad():
        for (x,y) in (loader):
            x=x.to(device=device)
            y=y.to(device=device)
            scores=model(x)
            scores=nn.functional.softmax(scores,dim=1)
            _,predictions=scores.max(1)
            correct+=(predictions==y).sum()
            total+= predictions.size(0)
    return (float(correct/total)*100)

In [ ]:
for a in tqdm(range(50)):
    for i,(data,target) in enumerate(train_dataloader):
        data=data.to(device)
        target=target.to(device)
        scores=model(data)
        loss=loss_fun(scores,target)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    writer.add_scalar('training accuracy',accuracy_check(train_dataloader,model),a)
    writer.add_scalar('testing accuracy',accuracy_check(test_dataloader,model),a)

            